In [1]:
using Feather
using LightGraphs
using GraphPlot
using Compose
using DataFrames
using Colors
using ProgressMeter
using Plots

In [2]:
df = Feather.read("save.feather")
size(df, 1)

23077780

In [3]:
bias = readtable("bias.csv", header=false)
code = r"^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?"
#print(bias)
c = 0
biasnames = String[]
for i in 1:size(bias,1)
    if ~isna(bias[i, 5])
        url = match(code, AbstractString(bias[i, 5]))
        bias[i, 5] = replace(url[4], "www.", "")
        push!(biasnames, bias[i, 5])
    else
        push!(biasnames, "NA")
    end
end

In [4]:
cats = Dict{String, Dict}()
for key in ["a", "link", "script", "img"]
    cats[key] = Dict{String, Dict}()
end

range = size(df, 1)

p = Progress(range, 0.1)
for i in 1:range
    sdom = get(df[i, 1])
    ddom = get(df[i, 2])
    if isascii(sdom) && isascii(ddom)
        cat = cats[get(df[i, 3])]
        if ~haskey(cat, sdom)
            cat[sdom] = Dict{String, Int}()
        end
        if ~haskey(cat[sdom], ddom)
            cat[sdom][ddom] = 1
        else
            cat[sdom][ddom] += 1
        end
    end
    next!(p)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:01:03days


In [5]:
mutdata = DataFrame(sdom=String[], ddom=String[], w=Int64[])
for src in keys(cats["a"])
    for dst in keys(cats["a"][src])
        if haskey(cats["a"], dst) && haskey(cats["a"][dst], src) && ~(src == dst)
            push!(mutdata, [src, dst, cats["a"][src][dst]])
        end
    end
end
size(mutdata, 1)
head(mutdata)

,sdom,ddom,w
1,bluebirdbanter.com,sbnation.com,13
2,gethampshire.co.uk,getsurrey.co.uk,2
3,securitytoday.com,ohsonline.com,3
4,sglinks.com,sgblogs.com,113
5,sglinks.com,classy.sg,17
6,gamrconnect.vgchartz.com,vgchartz.com,148


In [6]:
nodes = String[]
minsamp = 10
p = Progress(size(mutdata, 1), 1)
for i in 1:size(mutdata, 1)
    src = mutdata[i, 1]
    dst = mutdata[i, 2]
    if cats["a"][src][dst] > minsamp && cats["a"][dst][src] > minsamp
        if ~(src in nodes)
            push!(nodes, src)
        end
        if ~(dst in nodes)
            push!(nodes, dst)
        end
    end
    next!(p)
end

println("Number of node labels $(size(nodes,1))")

p = Progress(size(mutdata, 1), 1)
G = Graph(size(nodes,1))
for i in 1:size(mutdata, 1)
    src = mutdata[i, 1]
    dst = mutdata[i, 2]
    if src in nodes && dst in nodes
        add_edge!(G, find(nodes .== src)[1], find(nodes .== dst)[1])
    end
    next!(p)
end

println("Number of nodes $(nv(G))")
println("Number of edges $(ne(G))")


Number of node labels 1349


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


Number of nodes 1349
Number of edges 2767


In [7]:
nodesize = [length(neighbors(G, v)) for v in vertices(G)]
#print(nodesize)
function reality(x)
    if x in biasnames
        if isna(bias[find(biasnames .== x)[1], 4])
            return 2
        else
            return 3
        end
    end
    return 1
end


members = [reality(n) for n in nodes]
colors = [colorant"grey", colorant"blue", colorant"red"]
println("colors")
#graph = gplot(G, nodefillc=colors[members], nodelabel=nodes, nodelabelsize=nodesize*2)
println("graphed")
#draw(PNG("karate.png", 40cm, 40cm), graph)

colors
graphed


In [8]:
s2i = Dict{String, Int}()
i2s = Dict{Int, String}()
blacklist = String["twitter.com", "facebook.com", "youtube.com", "instagram.com", 
    "plus.google.com", "linkedin.com", "t.co", "itunes.apple.com", "pinterest.com", 
    "flickr.com", "bit.ly", "play.google.com"]
minsamp = 100
count = 0
body = cats["a"]
for src in keys(body)
    for dst in keys(body[src])
        if body[src][dst] > minsamp && haskey(cats["a"]~(src in blacklist) && ~(dst in blacklist)
            if ~haskey(s2i, src)
                count += 1
                s2i[src] = count
                i2s[count] = src
            end
            if ~haskey(s2i, dst)
                count += 1
                s2i[dst] = count
                i2s[count] = dst
            end
        end
    end
end

println("Number of node labels $(length(s2i))")

G = Graph(count)
for src in keys(cats["a"])
    for dst in keys(cats["a"][src])
        if cats["a"][src][dst] > minsamp && ~(src in blacklist) && ~(dst in blacklist)
            add_edge!(G, s2i[src], s2i[dst])
        end
    end
end

println("Number of nodes $(nv(G))")
println("Number of edges $(ne(G))")


Number of node labels 12869
Number of nodes 12869
Number of edges 14829


In [10]:
sg = sort(connected_components(G), by=size, rev=true)[1]

ss2i = Dict{String, Int}()
si2s = Dict{Int, String}()

c = 0
for i in sg
    c += 1
    ss2i[i2s[i]] = c
    si2s[c] = i2s[i]
end

H = Graph(size(sg, 1))

for j in 1:c
    src = si2s[j]
    if src in keys(cats["a"])
        for dst in keys(cats["a"][src])
            if cats["a"][src][dst] > minsamp && ~(dst in blacklist)
                add_edge!(H, j, ss2i[dst])
            end
        end
    end
end
lx, ly = spring_layout(H)


(process:27545): Pango-WARNING **: pango_layout_set_markup_with_accel: Error on line 1 char 31: Odd character '%', expected a '=' after attribute name 'search.scoop.co.nz' of element '%'


In [ ]:
sizes = [2 + log(length(all_neighbors(H, v))) for v in 1:c]
labels = [si2s[v] for v in 1:c]
members = [reality(si2s[v]) for v in 1:c]
colors = [colorant"grey", colorant"blue", colorant"red"]
draw(PNG("plots/beliefprop.png", 50cm, 50cm),
     gplot(H, lx, ly, nodelabel=labels, nodelabelsize=sizes, nodefillc=colors[members]))


In [11]:
using Iterators

function normalize_neighbors!(m, g::Graph, X::AbstractArray, i::Integer)
    Z = sum(m[i,neighbors(g,i), :])
    #@show i, Z
    for j in neighbors(g,i)
        for k in X
            m[i,j,k] /= Z
        end
    end
    return Z
end

function assert_nonzero(val)
    @assert val != 0 "val was zero"
    return val
end

function propogate!{T<:Real, U<:Integer}(m::AbstractArray{T,3},
                                         ϕ::AbstractArray{T,2},
                                         ψ::AbstractArray{T,2}, A, X::AbstractArray{U, 1})
    n = size(A, 1)
    for i in 1:n
        for j in neighbors(A,i)
            if j == i
                continue
            end
            for k in X
                m[i,j,k] = 0
                for l in X
                    w = ϕ[i,l] * ψ[l,k]
                    @assert w != NaN
                    sp = sum([p!=j ? log(assert_nonzero(m[p,i,l])) : 0 for p in neighbors(A, i)])
                    p = exp(sp) + 1e-16
                    if p == 0
                        warn("sp is $sp")
                        warn("p is 0! $i,$j,$k,$l")
                    end
                    @assert p != NaN
                    @assert m[j,i,l] != 0 "$i,$j,$l have m 0"
                    m[i,j,k] += w * p #/ m[j,i,l]
                end
            end
        end
        Z = normalize_neighbors!(m, A, X, i)
    end
    return m
end

function beliefs!{T<:Real}(m::AbstractArray{T}, b, ϕ, A, X)
    for i in 1:size(A,1)
        for k in X
            w = prod(T[m[p,i,k] for p in neighbors(A, i)])
            b[i,k] = ϕ[i,k] * w
            @assert b[i,k] >= 0 "Unnormalized beliefs are not all positive offending indices ($i, $k)=$(b[i,k])"
        end
        b[i,:] /= sum(b[i,:])
    end
    #@show b
    @assert all( 0.99 .< sum(b,2) .< 1.01) "Beliefs do not form a probability distributions"
    return b
end

type BeliefProblem{T<:Real, U<:Integer, G}
    m::AbstractArray{T,3}
    ϕ::AbstractArray{T,2}
    ψ::AbstractArray{T,2}
    A::G
    X::AbstractArray{U, 1}
end

type Messages{T<:Real, G} <: AbstractArray{T, 3}
    nrows::Int
    ncols::Int
    edges::Dict{Tuple{Int, Int}, Int}
    states::UnitRange
    storage::Array{T, 2}
end

function Messages(g::Graph, states::UnitRange)
    n=nv(g)
    d = Dict{Tuple{Int,Int}, Int}()
    i = 1
    for e in edges(g)
        d[(src(e), dst(e))] = i
        d[(dst(e), src(e))] = i+1
        i += 2
    end
    storage = ones(Float64, (i, length(states)))
    return Messages{Float64, Graph}(n,n,d, states, storage)
end

Base.size(m::Messages) = (m.nrows, m.ncols, length(m.states))
function Base.getindex(m::Messages, I::Vararg{Int})
    pair = I[1:2]
    if pair in keys(m.edges)
        return m.storage[m.edges[pair], I[3]]
    else
        throw(KeyError(pair))
    end
end

function Base.setindex!(m::Messages, v, I::Vararg{Int})
    pair = I[1:2]
    if pair in keys(m.edges)
        return m.storage[m.edges[pair], I[3]] = v
    else
        throw(KeyError(pair))
    end
end

logodds(b::AbstractMatrix) = log.(b[:,1]./b[:,2])

function beliefprop(g::Graph, ϕ, ψ, maxiter)
    n = size(g, 1)
    k = 2
    m = ones(Float64, (n,n,k))
    m = Messages(g, 1:k)
    #@show sort(collect(keys(m.edges)))
    pr = BeliefProblem(m, ϕ, ψ, g, 1:k)
    iterstates = []
    diffs = []
    i = 1
    m = nth(iterate(m->begin msg = propogate!(m, ϕ,ψ, g, 1:k);
            state = [maximum(msg.storage), minimum(msg.storage)]
            diff = i > 1 ? state-iterstates[i-1]:0;
            i+=1
            @show diff
            push!(iterstates, state)
            push!(diffs, diff)
            #msg.storage = max(msg.storage, 1e-60)
           return msg 
        end, m), maxiter)
    b = similar(pr.ϕ)
    beliefs!(pr.m, b, pr.ϕ, g, 1:k)
    lodds = logodds(b)
    #display(plotbeliefs(g, b[:,1]))
    display(bar(lodds, ylabel="Log Odds", xlabel="Vertex ID"))
    return pr, b, lodds
end

function plotbeliefs(g::Graph, b::AbstractVector)
    nodefillc=map(x-> RGB(1x, 0, 1-x), b./maximum(b))
    plo = gplot(H, lx, ly, nodelabel=[si2s[v] for v in 1:nv(g)], nodelabelsize=sizes, nodefillc=nodefillc)
    draw(PNG("plots/postbeliefprop.png", 50cm, 50cm), plo)
    return plo
    #return gplot(g, layout=(g)-> spring_layout(g; C=1), nodelabel=1:nv(g), nodelabelsize=1, nodefillc=nodefillc)
    #return gplot(g, layout=(g)-> begin pos = (200*begin Z=spring_layout(g); Z[1]end, log2(float(collect(1:nv(g))))); @show typeof(pos); return pos end, nodelabel=1:nv(g), nodelabelsize=1, nodefillc=nodefillc)
    #return gplot(g, layout=(g)-> (exp10(spectral_layout(g)[1]), log(float(vertices(g)))),nodelabel=1:nv(g), nodelabelsize=1, nodefillc=nodefillc)
end

function Psis(ϵ::Real)
    return [1-ϵ ϵ;
            ϵ 1-ϵ]
end

Psis (generic function with 1 method)

In [12]:
ϕ = ones((c,2))./2.0
for v in 1:c
    r = reality(si2s[v])
    if r == 3
        ϕ[v,:] = [0.99999, 0.01]
    elseif r == 2
        ϕ[v,:] = [0.1, 0.9]
    elseif r == 1
        ϕ[v,:] = [0.5, 0.5]
    end
end
println(nv(H))
println(minimum([length(all_neighbors(H, v)) for v in 1:c]))

8205
1


In [13]:
pr, b, lodds = beliefprop(H, ϕ, Psis(0.44), 10);

diff = 0
diff = [0.0, 2.26e-17]
diff = [0.0, 7.39557e-32]
diff = [0.0, 1.84889e-32]
diff = [0.0, -1.2326e-32]
diff = [0.0, 1.2326e-32]
diff = [0.0, 0.0]
diff = [0.0, -1.2326e-32]
diff = [0.0, 1.2326e-32]
diff = [0.0, 0.0]


LoadError: [91mAssertionError: Beliefs do not form a probability distributions[39m

In [9]:
sgs = connected_components(G)
sort!(sgs, by=size, rev=true)
print([size(sg, 1) for sg in sgs[1:50]])
H = Graph(2)
r = 5
p = Progress(r, 1)
plots = []
k = 1
for sg in sgs[1:r]
    #Subgraph String-To-Int, Int-To-String
    ss2i = Dict{String, Int}()
    si2s = Dict{Int, String}()
    c = 0
    for i in sg
        c += 1
        ss2i[i2s[i]] = c
        si2s[c] = i2s[i]
    end
    H = Graph(size(sg, 1))
    for j in 1:c
        src = si2s[j]
        if src in keys(cats["a"])
            for dst in keys(cats["a"][src])
                if cats["a"][src][dst] > minsamp && ~(dst in blacklist)
                    add_edge!(H, j, ss2i[dst])
                end
            end
        end
    end
    sizes = [log(length(all_neighbors(H, v))) for v in 1:c]
    labels = [si2s[v] for v in 1:c]
    members = [reality(si2s[v]) for v in 1:c]
    colors = [colorant"grey", colorant"blue", colorant"red"]
    draw(PNG("plots/plot"*string(k)*".png", 50cm, 50cm), gplot(H, nodelabel=labels, nodelabelsize=sizes, nodefillc=colors[members]))
    k += 1
    next!(p)
end


[8205, 384, 69, 60, 34, 33, 33, 30, 29, 28, 27, 26, 26, 26, 25, 25, 24, 24, 24, 23, 22, 20, 19, 19, 19, 19, 18, 17, 17, 17, 17, 16, 16, 16, 16, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 13, 12, 12, 12]


(process:27545): Pango-WARNING **: pango_layout_set_markup_with_accel: Error on line 1 char 31: Odd character '%', expected a '=' after attribute name 'search.scoop.co.nz' of element '%'
Progress: 100%|█████████████████████████████████████████| Time: 0:04:06
